In [ ]:
import pandas as pd
from io import StringIO
import numpy as np

In [ ]:
path='/content/drive/MyDrive/Brevant/Brevant_Rabi_15dayFarmMeanNDVI.csv'
data=pd.read_csv(path)
data.columns

Index(['system:index', '2021-11-01_NDVI', '2021-11-16_NDVI', '2021-12-01_NDVI',
       '2021-12-16_NDVI', '2021-12-31_NDVI', '2022-01-15_NDVI',
       '2022-01-30_NDVI', '2022-02-14_NDVI', '2022-03-01_NDVI',
       '2022-03-16_NDVI', '2022-03-31_NDVI', '2022-04-15_NDVI',
       '2022-04-30_NDVI', '2022-05-15_NDVI', '2022-05-30_NDVI', 'ID',
       'Percentage_crop_Area', 'block', 'district', 'farmarea', 'farmer',
       'mobile', 'pincode', 'state', 'village', '.geo'],
      dtype='object')

In [ ]:
data['ID']

0      439
1      424
2       11
3       49
4      327
      ... 
649    467
650    470
651    568
652    319
653    320
Name: ID, Length: 654, dtype: int64

In [ ]:
date_n=[ '2021-11-01_NDVI','2021-11-16_NDVI', '2021-12-01_NDVI',
       '2021-12-16_NDVI', '2021-12-31_NDVI', '2022-01-15_NDVI',
       '2022-01-30_NDVI', '2022-02-14_NDVI', '2022-03-01_NDVI',
       '2022-03-16_NDVI', '2022-03-31_NDVI', '2022-04-15_NDVI',
       '2022-04-30_NDVI', '2022-05-15_NDVI', '2022-05-30_NDVI']
dates=[x[:-5] for x in date_n]

d=data[date_n]
d.columns=dates
d

,2021-11-01,2021-11-16,2021-12-01,2021-12-16,2021-12-31,2022-01-15,2022-01-30,2022-02-14,2022-03-01,2022-03-16,2022-03-31,2022-04-15,2022-04-30,2022-05-15,2022-05-30
0,0.383763,0.252958,0.128371,0.111587,0.236452,0.246672,0.386560,0.405403,0.436236,0.189748,0.113964,0.093979,0.118796,0.408203,NaN
1,0.089952,0.026709,0.109441,0.200751,0.510067,0.493535,0.418388,0.436998,0.437386,0.329852,0.249011,0.225630,0.119636,0.095890,0.082316
2,0.402220,0.272680,0.162283,0.092198,NaN,0.241270,0.399494,0.427597,0.401356,0.203238,0.121927,0.103995,0.108179,0.181779,0.218903
3,0.424080,0.291746,0.187179,0.116489,NaN,0.161314,0.245974,0.340671,0.440934,0.316175,0.148545,0.121026,0.126117,0.158231,0.183571
4,0.349592,0.178932,0.128322,0.102034,NaN,0.305057,0.330224,0.353620,0.319644,0.171745,0.111111,0.101831,0.106771,0.234713,0.257654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,0.212094,0.170968,0.168588,0.064713,NaN,NaN,0.351430,0.418818,0.460551,0.285315,0.103379,0.134114,0.232608,0.109882,NaN
650,0.219707,0.136174,0.165992,0.039437,NaN,NaN,0.327689,0.389864,0.422877,0.230226,0.094878,0.118777,0.096836,0.093652,NaN
651,0.447358,0.376030,0.440122,0.365668,NaN,NaN,0.181727,0.216403,0.264619,0.303663,0.244447,0.279130,0.304836,0.223794,NaN
652,0.331257,0.174342,0.114448,0.165502,NaN,NaN,0.297289,0.336019,0.326942,0.158943,0.084987,0.090954,0.087457,0.089918,NaN


In [ ]:
#  Max value between jan-march
MAX_after=4

In [ ]:
Date_index=np.arange(0,len(dates),1)
dictn={dates[i]:Date_index[i] for i in range(len(Date_index))}

def calc_maxval(s):
    assert isinstance(s, pd.Series)
    return s[MAX_after :].max()

def calc_max_date(s):
    return s[MAX_after :].argmax()+MAX_after

def Sowing(s):
    try:
        left= s[ : s[MAX_after :].argmax()+MAX_after+1]
        sorted_values=left[left.argmin():].sort_values()
        f= sorted_values[sorted_values>=0.2]
        sowing=s.keys()[dictn[f.keys()[0]]-1]
    except:
        sowing = None
    return sowing

def Harvesting(s):
    try:
        right= s[s[MAX_after :].argmax()+MAX_after:]
        sorted_values=right.sort_values(ascending = False)
        f= sorted_values[sorted_values<=0.2]
        harvesting=f.keys()[0]
    except:
        harvesting = None
    return harvesting



In [ ]:
max_val = d.apply(lambda x: calc_maxval(x), axis=1).rename('MaxVal')
max_date = d.apply(lambda x: calc_max_date(x), axis=1).rename('max_date')

sow_date = d.apply(lambda x: Sowing(x), axis=1).rename('Sowing_FN')

harv_date = d.apply(lambda x: Harvesting(x), axis=1).rename('Harvesting_FN')

SOW_Harvest = pd.concat([max_val, max_date, sow_date, harv_date], axis=1)
SOW_Harvest

,MaxVal,max_date,Sowing_FN,Harvesting_FN
0,0.436236,8,2021-12-16,2022-03-16
1,0.510067,4,2021-12-01,2022-04-30
2,0.427597,7,2021-12-31,2022-05-15
3,0.440934,8,2022-01-15,2022-05-30
4,0.353620,7,2021-12-31,2022-03-16
...,...,...,...,...
649,0.460551,8,2022-01-15,2022-04-15
650,0.422877,8,2022-01-15,2022-04-15
651,0.304836,12,2022-01-30,None
652,0.336019,7,2022-01-15,2022-03-16


# Maize health

In [ ]:
max_ndvi_df=pd.read_csv('/content/drive/MyDrive/Brevant/Brevant_Rabi_Max_NDVI_Mean_Farm.csv')
max_ndvi_df

,system:index,ID,Percentage_crop_Area,block,district,farmarea,farmer,mean,mobile,pincode,state,village,.geo
0,000000000000000001b6,439,100.0,Belhar,Banka,948.08,Pawan Kumar tanti,0.442472,7827963678,813202,Bihar,Tola Hariari,"{""type"":""Polygon"",""coordinates"":[[[86.59440474..."
1,000000000000000001a7,424,100.0,Dinapur-Cum-Khagaul,Patna,465.24,lalan mahato,0.510067,7781001694,841217,Bihar,Kedalpura,"{""type"":""Polygon"",""coordinates"":[[[85.08566343..."
2,0000000000000000000a,11,100.0,Bikram,Patna,2001.76,Awadh Bihari Singh,0.430624,9608071874,801104,Bihar,Gona,"{""type"":""Polygon"",""coordinates"":[[[84.79036837..."
3,00000000000000000030,49,100.0,Bikram,Patna,3235.06,Amlendu Bhushan,0.447565,9006460939,801104,Bihar,Datiana,"{""type"":""Polygon"",""coordinates"":[[[84.86897789..."
4,00000000000000000146,327,100.0,Bikram,Patna,570.99,kaushal prashad,0.367256,6204669056,801104,Bihar,Wazirpur,"{""type"":""Polygon"",""coordinates"":[[[84.82292871..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,000000000000000001d2,467,100.0,Bairia,Pashchim Champaran,1418.64,Suresh Sah,0.460551,7632843804,845459,Bihar,Nautan Dube,"{""type"":""Polygon"",""coordinates"":[[[84.49169335..."
650,000000000000000001d5,470,100.0,Bairia,Pashchim Champaran,1556.57,Ajay Kumar Ray,0.422877,9931965648,845459,Bihar,Machharganwa,"{""type"":""Polygon"",""coordinates"":[[[84.48317199..."
651,00000000000000000237,568,100.0,Nautan,Pashchim Champaran,909.87,Ashok Prasad,0.464607,8051304754,845459,Bihar,Tola Halkhorwa,"{""type"":""Polygon"",""coordinates"":[[[84.57194373..."
652,0000000000000000013e,319,100.0,Mainatanr,Pashchim Champaran,1566.02,Sagir Ji Majaria,0.346813,7870027680,845306,Bihar,Bhaluahia,"{""type"":""Polygon"",""coordinates"":[[[84.60850395..."


# New section

*   NDVI<0.4 Very low
*   0.4<=NDVI<0.5 low
*   o.5<=NDVI<0.6 Normal
*   0.6<=NDVI<0.7 Good
*   NDVI>0.7 V.good


In [ ]:
ndvi_Conditions = [
    (max_ndvi_df['mean'] < 0.4),
    (max_ndvi_df['mean'] >=0.4) & (max_ndvi_df['mean'] < 0.5),
    (max_ndvi_df['mean'] >=0.5) & (max_ndvi_df['mean'] < 0.6),
    (max_ndvi_df['mean'] >=0.6) & (max_ndvi_df['mean'] < 0.7),
    (max_ndvi_df['mean'] >=0.7)
]
health_Categories = ['Very low', 'Low', 'Normal','Good','Very good']
max_ndvi_df['Maize_health'] = np.select(ndvi_Conditions, health_Categories)
max_ndvi_df

,system:index,ID,Percentage_crop_Area,block,district,farmarea,farmer,mean,mobile,pincode,state,village,.geo,Maize_health
0,000000000000000001b6,439,100.0,Belhar,Banka,948.08,Pawan Kumar tanti,0.442472,7827963678,813202,Bihar,Tola Hariari,"{""type"":""Polygon"",""coordinates"":[[[86.59440474...",Low
1,000000000000000001a7,424,100.0,Dinapur-Cum-Khagaul,Patna,465.24,lalan mahato,0.510067,7781001694,841217,Bihar,Kedalpura,"{""type"":""Polygon"",""coordinates"":[[[85.08566343...",Normal
2,0000000000000000000a,11,100.0,Bikram,Patna,2001.76,Awadh Bihari Singh,0.430624,9608071874,801104,Bihar,Gona,"{""type"":""Polygon"",""coordinates"":[[[84.79036837...",Low
3,00000000000000000030,49,100.0,Bikram,Patna,3235.06,Amlendu Bhushan,0.447565,9006460939,801104,Bihar,Datiana,"{""type"":""Polygon"",""coordinates"":[[[84.86897789...",Low
4,00000000000000000146,327,100.0,Bikram,Patna,570.99,kaushal prashad,0.367256,6204669056,801104,Bihar,Wazirpur,"{""type"":""Polygon"",""coordinates"":[[[84.82292871...",Very low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,000000000000000001d2,467,100.0,Bairia,Pashchim Champaran,1418.64,Suresh Sah,0.460551,7632843804,845459,Bihar,Nautan Dube,"{""type"":""Polygon"",""coordinates"":[[[84.49169335...",Low
650,000000000000000001d5,470,100.0,Bairia,Pashchim Champaran,1556.57,Ajay Kumar Ray,0.422877,9931965648,845459,Bihar,Machharganwa,"{""type"":""Polygon"",""coordinates"":[[[84.48317199...",Low
651,00000000000000000237,568,100.0,Nautan,Pashchim Champaran,909.87,Ashok Prasad,0.464607,8051304754,845459,Bihar,Tola Halkhorwa,"{""type"":""Polygon"",""coordinates"":[[[84.57194373...",Low
652,0000000000000000013e,319,100.0,Mainatanr,Pashchim Champaran,1566.02,Sagir Ji Majaria,0.346813,7870027680,845306,Bihar,Bhaluahia,"{""type"":""Polygon"",""coordinates"":[[[84.60850395...",Very low


In [ ]:
final=pd.concat([max_ndvi_df,SOW_Harvest],axis=1)
final

,system:index,ID,Percentage_crop_Area,block,district,farmarea,farmer,mean,mobile,pincode,state,village,.geo,Maize_health,MaxVal,max_date,Sowing_FN,Harvesting_FN
0,000000000000000001b6,439,100.0,Belhar,Banka,948.08,Pawan Kumar tanti,0.442472,7827963678,813202,Bihar,Tola Hariari,"{""type"":""Polygon"",""coordinates"":[[[86.59440474...",Low,0.436236,8,2021-12-16,2022-03-16
1,000000000000000001a7,424,100.0,Dinapur-Cum-Khagaul,Patna,465.24,lalan mahato,0.510067,7781001694,841217,Bihar,Kedalpura,"{""type"":""Polygon"",""coordinates"":[[[85.08566343...",Normal,0.510067,4,2021-12-01,2022-04-30
2,0000000000000000000a,11,100.0,Bikram,Patna,2001.76,Awadh Bihari Singh,0.430624,9608071874,801104,Bihar,Gona,"{""type"":""Polygon"",""coordinates"":[[[84.79036837...",Low,0.427597,7,2021-12-31,2022-05-15
3,00000000000000000030,49,100.0,Bikram,Patna,3235.06,Amlendu Bhushan,0.447565,9006460939,801104,Bihar,Datiana,"{""type"":""Polygon"",""coordinates"":[[[84.86897789...",Low,0.440934,8,2022-01-15,2022-05-30
4,00000000000000000146,327,100.0,Bikram,Patna,570.99,kaushal prashad,0.367256,6204669056,801104,Bihar,Wazirpur,"{""type"":""Polygon"",""coordinates"":[[[84.82292871...",Very low,0.353620,7,2021-12-31,2022-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,000000000000000001d2,467,100.0,Bairia,Pashchim Champaran,1418.64,Suresh Sah,0.460551,7632843804,845459,Bihar,Nautan Dube,"{""type"":""Polygon"",""coordinates"":[[[84.49169335...",Low,0.460551,8,2022-01-15,2022-04-15
650,000000000000000001d5,470,100.0,Bairia,Pashchim Champaran,1556.57,Ajay Kumar Ray,0.422877,9931965648,845459,Bihar,Machharganwa,"{""type"":""Polygon"",""coordinates"":[[[84.48317199...",Low,0.422877,8,2022-01-15,2022-04-15
651,00000000000000000237,568,100.0,Nautan,Pashchim Champaran,909.87,Ashok Prasad,0.464607,8051304754,845459,Bihar,Tola Halkhorwa,"{""type"":""Polygon"",""coordinates"":[[[84.57194373...",Low,0.304836,12,2022-01-30,None
652,0000000000000000013e,319,100.0,Mainatanr,Pashchim Champaran,1566.02,Sagir Ji Majaria,0.346813,7870027680,845306,Bihar,Bhaluahia,"{""type"":""Polygon"",""coordinates"":[[[84.60850395...",Very low,0.336019,7,2022-01-15,2022-03-16


In [ ]:
final.columns

Index(['system:index', 'ID', 'Percentage_crop_Area', 'block', 'district',
       'farmarea', 'farmer', 'mean', 'mobile', 'pincode', 'state', 'village',
       '.geo', 'Maize_health', 'MaxVal', 'max_date', 'Sowing_FN',
       'Harvesting_FN'],
      dtype='object')

In [ ]:
final=final[['ID','state','district', 'village', 'block','farmer', 'mobile', 'pincode',  'farmarea',
       'Maize_health','Sowing_FN', 'Harvesting_FN','Percentage_crop_Area','.geo']]

In [ ]:
final

,ID,state,district,village,block,farmer,mobile,pincode,farmarea,Maize_health,Sowing_FN,Harvesting_FN,Percentage_crop_Area,.geo
0,439,Bihar,Banka,Tola Hariari,Belhar,Pawan Kumar tanti,7827963678,813202,948.08,Low,2021-12-16,2022-03-16,100.0,"{""type"":""Polygon"",""coordinates"":[[[86.59440474..."
1,424,Bihar,Patna,Kedalpura,Dinapur-Cum-Khagaul,lalan mahato,7781001694,841217,465.24,Normal,2021-12-01,2022-04-30,100.0,"{""type"":""Polygon"",""coordinates"":[[[85.08566343..."
2,11,Bihar,Patna,Gona,Bikram,Awadh Bihari Singh,9608071874,801104,2001.76,Low,2021-12-31,2022-05-15,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.79036837..."
3,49,Bihar,Patna,Datiana,Bikram,Amlendu Bhushan,9006460939,801104,3235.06,Low,2022-01-15,2022-05-30,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.86897789..."
4,327,Bihar,Patna,Wazirpur,Bikram,kaushal prashad,6204669056,801104,570.99,Very low,2021-12-31,2022-03-16,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.82292871..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,467,Bihar,Pashchim Champaran,Nautan Dube,Bairia,Suresh Sah,7632843804,845459,1418.64,Low,2022-01-15,2022-04-15,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.49169335..."
650,470,Bihar,Pashchim Champaran,Machharganwa,Bairia,Ajay Kumar Ray,9931965648,845459,1556.57,Low,2022-01-15,2022-04-15,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.48317199..."
651,568,Bihar,Pashchim Champaran,Tola Halkhorwa,Nautan,Ashok Prasad,8051304754,845459,909.87,Low,2022-01-30,None,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.57194373..."
652,319,Bihar,Pashchim Champaran,Bhaluahia,Mainatanr,Sagir Ji Majaria,7870027680,845306,1566.02,Very low,2022-01-15,2022-03-16,100.0,"{""type"":""Polygon"",""coordinates"":[[[84.60850395..."


In [ ]:
final.to_csv('/content/drive/MyDrive/Brevant/Final_results/Sow_harvest_Health_bihar_maize.csv',index=False)